# Process data & Store into Redis

In [235]:
import json
import os

## process data
Select the attributes to be stored.

In [236]:
# selected attributes in tweets
twt_attributes = ['created_at',
                       'id_str',
                       'text',
                       'in_reply_to_status_id_str',
                       'in_reply_to_user_id_str',
                       'in_reply_to_screen_name',
                       'quoted_status_id_str',
                       'quoted_status',
                       'quote_count',
                       'reply_count',
                       'retweet_count',
                       'favorite_count',
                       'entities',
                       'extended_entities',
                       'user.id_str',
                       'user.name',
                       'user.screen_name',
                       'user.url',
                       'user.description',
                       'user.protected',
                       'user.verified',
                       'user.followers_count',
                       'user.friends_count',
                       'user.listed_count',
                       'user.favourites_count',
                       'user.statuses_count',
                       'user.created_at'
                      ]
len(twt_attributes) # 27

27

In [237]:
# so the selected retweets attributes has an extra "retweeted_status.id_str"
retwt_attributes = ['created_at',
                    'id_str',
                    'text',
                    'in_reply_to_status_id_str',
                    'in_reply_to_user_id_str',
                    'in_reply_to_screen_name',
                    'quoted_status_id_str',
                    'quoted_status',
                    'quote_count',
                    'reply_count',
                    'retweet_count',
                    'favorite_count',
                    'entities',
                    'extended_entities',
                    'retweeted_status.id_str',
                    'user.id_str',
                    'user.name',
                    'user.screen_name',
                    'user.url',
                    'user.description',
                    'user.protected',
                    'user.verified',
                    'user.followers_count',
                    'user.friends_count',
                    'user.listed_count',
                    'user.favourites_count',
                    'user.statuses_count',
                    'user.created_at']
len(retwt_attributes) # 28

28

In [238]:
# selected attributes in user
user_attr_tracking = ['id_str',
                      'name',
                      'screen_name',
                      'url',
                      'description',
                      'protected',
                      'verified',
                      'followers_count',
                      'friends_count',
                      'listed_count',
                      'favourites_count',
                      'statuses_count',
                      'created_at']

len(user_attr_tracking) #13

13

### user_store function

In [239]:
def user_store(user, userid_table):
    
    # if user has not been seen before, add id to userid_table
    if user['id_str'] not in userid_table:
        userid_table.append(user['id_str'])
        
        # update datastores with defined tracking attributes
        user_attr_selected = {key: user[key] for key in user_attr_tracking if key in user}
        user_list.append(user_attr_selected)

In [240]:
# test test

twt_id_record = []
userid_table = [] # store userid
user_list = [] # store every attributes of user needed tracking

with open("./data/corona-out-2", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            if data["id_str"] not in twt_id_record:
                twt_id_record.append(data["id_str"])
            # if tweet has been seen before 
            else:
                # ignore this tweet, go to next iteration of loop
                continue
            
            user = data['user']
            user_store(user, userid_table)
        except:
            continue

In [241]:
len(user_list)

15505

### tweets / retweet store

In [242]:
# this is a partial template for processing the dataset to store into the datastores.
# you will need to write functions for the psuedocode sections and invoke them here

#insert path and replace name of the file below as needed

twt_id_record = []
userid_table = [] # store userid
user_list = [] # store tracking attributes of user
retwt_list = [] # list to store retweets
retwt_lines = 0 # count # of retweets
twt_list = []
twt_lines = 0

with open("./data/corona-out-2", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            if data["id_str"] not in twt_id_record:
                twt_id_record.append(data["id_str"])
            # if tweet has been seen before 
            else:
                # ignore this tweet, go to next iteration of loop
                continue
            
            user = data['user']
            user_store(user, userid_table)
            
            
            if ( data['text'].startswith('RT') ):
                # try:
                    # update retweet information
                    selected_dict = {}
                    retwt_lines = retwt_lines + 1
                    for attribute in retwt_attributes:
                        try:
                            # assigns the reference: any changes made to one will affect the other
                            current_dict = data
                            current_selected_dict = selected_dict
                            # parent-child attributes
                            for sub_attribute in attribute.split('.'):
                                # parent attributes
                                current_dict = current_dict[sub_attribute]
                                # child attributes
                                if sub_attribute == attribute.split('.')[-1]:
                                    current_selected_dict[sub_attribute] = current_dict
                                else:
                                    if sub_attribute not in current_selected_dict:
                                        current_selected_dict[sub_attribute] = {}
                                    current_selected_dict = current_selected_dict[sub_attribute]
                        except:
                            continue
                    retwt_list.append(selected_dict)
                # except:
                    # continue

                # pass  # does nothing right now
                
            else:
                # add the new tweet to datastore
                    selected_dict = {}
                    twt_lines = twt_lines + 1
                    for attribute in twt_attributes:
                        try:
                            # assigns the reference: any changes made to one will affect the other
                            current_dict = data
                            current_selected_dict = selected_dict
                            # parent-child attributes
                            for sub_attribute in attribute.split('.'):
                                # parent attributes
                                current_dict = current_dict[sub_attribute]
                                # child attributes
                                if sub_attribute == attribute.split('.')[-1]:
                                    current_selected_dict[sub_attribute] = current_dict
                                else:
                                    if sub_attribute not in current_selected_dict:
                                        current_selected_dict[sub_attribute] = {}
                                    current_selected_dict = current_selected_dict[sub_attribute]
                        except:
                            continue
                    twt_list.append(selected_dict)
        except:
            continue

In [243]:
len(userid_table)

15505

In [244]:
len(user_list)

15505

In [245]:
user_list[1]

{'id_str': '1225145123920588805',
 'name': 'efe09',
 'screen_name': 'efe0927183508',
 'url': None,
 'description': "Allah'ın en değerli eseri insandır.\nCanı yanan sabretsin.\nCan yakan, canının yanacağı günü beklesin..\n677 khk\nRT düşüncem olduğu anlamına gelmez bilgi amaçlıdır",
 'protected': False,
 'verified': False,
 'followers_count': 653,
 'friends_count': 983,
 'listed_count': 0,
 'favourites_count': 1255,
 'statuses_count': 4177,
 'created_at': 'Wed Feb 05 19:52:38 +0000 2020'}

In [246]:
twt_lines

7349

In [247]:
len(twt_list)

7349

In [248]:
twt_list[1]

{'created_at': 'Sun Apr 12 18:27:26 +0000 2020',
 'id_str': '1249403771261722624',
 'text': '#Bo der Osterhase hat nicht Corona, er ist gekommen.',
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'favorite_count': 0,
 'entities': {'hashtags': [{'text': 'Bo', 'indices': [0, 3]}],
  'urls': [],
  'user_mentions': [],
  'symbols': []},
 'user': {'id_str': '1171484224244744198',
  'name': 'Elfpunkt',
  'screen_name': 'meElfpunkt',
  'url': None,
  'description': 'Mal 1 mal 0 mal Tilt',
  'protected': False,
  'verified': False,
  'followers_count': 3,
  'friends_count': 53,
  'listed_count': 0,
  'favourites_count': 72,
  'statuses_count': 60,
  'created_at': 'Tue Sep 10 18:03:17 +0000 2019'}}

In [249]:
retwt_lines

11157

In [250]:
len(retwt_list)

11157

In [251]:
retwt_list[1]

{'created_at': 'Sun Apr 12 18:27:25 +0000 2020',
 'id_str': '1249403768023678982',
 'text': 'RT @lale_karanfil: In Turkey, there are 300 thousand prisoners and 150 thousand prison employees in prisons. \nPrisons are the most risky pl…',
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'favorite_count': 0,
 'entities': {'hashtags': [],
  'urls': [],
  'user_mentions': [{'screen_name': 'lale_karanfil',
    'name': 'Karanfil Lale',
    'id': 1087735689091928064,
    'id_str': '1087735689091928064',
    'indices': [3, 17]}],
  'symbols': []},
 'retweeted_status': {'id_str': '1249397541596286979'},
 'user': {'id_str': '1225145123920588805',
  'name': 'efe09',
  'screen_name': 'efe0927183508',
  'url': None,
  'description': "Allah'ın en değerli eseri insandır.\nCanı yanan sabretsin.\nCan yakan, canının yanacağı günü beklesin..\n677 khk\nRT düşüncem olduğu anlamına gelmez bilgi 

In [252]:
# # # encapsulation test: don't know why it didn't work as a function

# # store tweets/retweets data & count nums
# def retwt_store(data):
#     # list for store tweets/retweets data
#     selected_dict = {}
#     retwt_lines = retwt_lines + 1
#     for attribute in retwt_attributes:
#         try:
#             # assigns the reference: any changes made to one will affect the other
#             current_dict = data
#             current_selected_dict = selected_dict
#             # parent-child attributes
#             for sub_attribute in attribute.split('.'):
#                 # parent attributes
#                 current_dict = current_dict[sub_attribute]
#                 # child attributes
#                 if sub_attribute == attribute.split('.')[-1]:
#                     current_selected_dict[sub_attribute] = current_dict
#                 else:
#                     if sub_attribute not in current_selected_dict:
#                         current_selected_dict[sub_attribute] = {}
#                     current_selected_dict = current_selected_dict[sub_attribute]
#         except:
#             continue
#     retwt_list.append(selected_dict)

## Store in Redis (test: user_list only)

In [253]:
import redis # for cache
import pymongo # for NoSQL data store

from redis.commands.json.path import Path
import redis.commands.search.aggregation as aggregations
import redis.commands.search.reducers as reducers
from redis.commands.search.field import TextField, NumericField, TagField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import NumericFilter, Query

# connect to a Redis cluster
from redis.cluster import RedisCluster
from string import ascii_letters
import random

In [254]:
redis_client = redis.Redis(host='localhost', port=6379, decode_responses=True)
redis_pool = redis.ConnectionPool(host='localhost', port=6379, decode_responses=True)

In [255]:
# initiate: remove all the data exists.
for key in redis_client.keys():
    redis_client.delete(key)

In [257]:
# check keys in the Redis database
len(redis_client.keys())

0

In [258]:
import datetime
from typing import List, Optional
from pydantic import BaseModel, NonNegativeInt, HttpUrl, AnyUrl, ValidationError
from redis_om import (EmbeddedJsonModel, Field, JsonModel, HashModel, Migrator, NotFoundError, get_redis_connection)

[Field Types supported by Pydantic](https://docs.pydantic.dev/usage/types/)

In [259]:
# del twt_user_model

In [260]:
# build an embedded sub-model twt_user with Redis OM

class twt_user_model(EmbeddedJsonModel):
    id_str: str = Field(index=True)
    name: str = Field(index=True, full_text_search=True)
    screen_name: str = Field(index=True, full_text_search=True)
    url: Optional[AnyUrl]
    description: Optional[str] = Field(index=False, full_text_search=True) # perform full text searches on the values
    protected: Optional[bool]
    verified: Optional[bool]
    followers_count: Optional[NonNegativeInt] = Field(index=True)
    friends_count: Optional[NonNegativeInt] = Field(index=True)
    listed_count: Optional[NonNegativeInt]
    favourites_count: Optional[NonNegativeInt]
    statuses_count: Optional[NonNegativeInt] = Field(index=True)
    created_at: str = Field(index=True, full_text_search=True) # don't really know how to define this type of data yet, got errors on datetime.datetime
    
    class Meta:
        # A redis.asyncio.Redis or redis.Redis client instance that the model will use to communicate with Redis.
        database = get_redis_connection()

In [261]:
# store user_list data into Redis, using "id_str" as key

for i in range(len(user_list)):
    new_user = twt_user_model(**user_list[i])
    new_user.save()
    new_user.expire(24 * 3600) # time to live: 1 day

In [262]:
# use Migrator to create the indexes for any models that have indexed fields
Migrator().run()

In [263]:
# check storage status: should be 15505
len(redis_client.keys())

15508

### search

In [264]:
# search by pk  - .get(pk)

twt_user_model.get("01GYGY7N04C7PGGXRQXW10X6VX").dict()

{'pk': '01GYGY7N04C7PGGXRQXW10X6VX',
 'id_str': '719847091493847040',
 'name': '両棲装〇戦闘車太郎',
 'screen_name': 'd2N5Q4GciZtsa2e',
 'url': None,
 'description': '自称：軍板泡沫コテにしてカエルの成虫、あるいはツイレイシスト≡ポリコレという怪物と戦う為に自らも怪物となることを選んだ両棲類。\u3000\n無言フォロー・リムーブは当然の権利、キミもオレも行使するべき。\u3000\n\u3000\n#とある魔術の電脳戦機 出戻りますた、ファニーランチャーもっと中って？',
 'protected': False,
 'verified': False,
 'followers_count': 1165,
 'friends_count': 4972,
 'listed_count': 24,
 'favourites_count': 223036,
 'statuses_count': 309045,
 'created_at': 'Tue Apr 12 11:18:15 +0000 2016'}

In [265]:
# search by id

twt_user_model.find(twt_user_model.id_str == "772526540240347136").all()

[twt_user_model(pk='01GYGY8B5W717BBR75PX80174Q', id_str='772526540240347136', name='Gabriela', screen_name='KeniaG2811', url=None, description='26.03 👫❤ @Flamengo @Cristiano', protected=False, verified=False, followers_count=656, friends_count=1360, listed_count=0, favourites_count=19642, statuses_count=11110, created_at='Sun Sep 04 20:07:34 +0000 2016')]

In [266]:
# search by str

twt_user_model.find(twt_user_model.screen_name == "JusticiarThe").all()

[twt_user_model(pk='01GYGY7VN2BEPZKRTHT4ZN4PBT', id_str='1248347704197025792', name='Tim Gustafsson 🇸🇪🇳🇴🇩🇰🇫🇮🇲🇫🇩🇪🇪🇺🇬🇧', screen_name='JusticiarThe', url=None, description='17 years experience in medicines', protected=False, verified=False, followers_count=15, friends_count=159, listed_count=0, favourites_count=52, statuses_count=311, created_at='Thu Apr 09 20:31:13 +0000 2020')]

In [267]:
# search by some number: tweeter with the mamixmum number of followers

top_tweeter = twt_user_model.find(
    twt_user_model.followers_count >= 2000000
).sort_by("-followers_count").all()

In [268]:
for i in range(len(top_tweeter)):
    print(top_tweeter[i].screen_name + "\t" + str(top_tweeter[i].followers_count) + "\n")

detikcom	15884929

la_patilla	7121282

skynewsarabia	5652410

virsanghvi	4329132

NTelevisa_com	3047122

repubblica	2992272

casspernyovest	2702055

